<a href="https://colab.research.google.com/github/akshat2635/Auto-Eval-24/blob/main/AutoEval_final_model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# installing required libraries
!pip install easyocr fuzzywuzzy pytesseract
!sudo apt install tesseract-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [6]:
# importing required modules
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import shutil
import PIL.Image
from PIL import Image

from fuzzywuzzy import fuzz

import easyocr
from pylab import rcParams
from IPython.display import Image

import cv2
import pytesseract
from pytesseract import Output
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

ModuleNotFoundError: No module named 'fuzzywuzzy'

In [5]:
# initialized easyOCR reader with language=english
reader = easyocr.Reader(['en'],gpu = True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [6]:
#
def txt(text):
    str1 = 'true'
    str2 = 'false'
    str3=text.lower()
    # print(str3)
    r1=fuzz.ratio(str1, str3)
    r2=fuzz.ratio(str2, str3)
    if(r1>r2):
        return str1
    else:
        return str2

def preprocess_image(img):

    # Apply thresholding to binarize the image
    if len(img.shape) == 3:  # Check if the image has 3 channels (color image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to enhance the text
    kernel = np.ones((3, 3), np.uint8)
    morphed_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

    return morphed_img

def get_center_part(image_path):
    """
    Extracts the center part of the image when divided into 16 parts.
    """
    # Load the image
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Calculate the dimensions of each part
    part_height = height // 8
    part_width = width // 8

    # Calculate the coordinates of the center part
    start_y = part_height
    end_y = 3 * part_height
    start_x = part_width
    end_x = 3 * part_width

    # Extract the center part
    center_part = image[start_y:end_y, start_x:end_x]

    return center_part
def get_image_orientation(image_path):
    """
    Detect the orientation of the text in the image and return the angle.
    """
    # image = cv2.imread(image_path)
    image=get_center_part(image_path)
    height, width = image.shape[:2]
    # image = image[:height//2, :width//2]
    # height, width = image.shape[:2]
    image=cv2.resize(image,(height//3,width//3))
    # image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # Use Tesseract to detect text orientation
    osd = pytesseract.image_to_osd(image, output_type=Output.DICT)
    angle = osd['rotate']
    return -angle

def correct_image_orientation(image_path):
    """
    Correct the orientation of the image and save it.
    """
    angle = get_image_orientation(image_path)
    image = cv2.imread(image_path)
    if angle == 0:
        return image

    # print(f"Rotating image {image_path} by {angle} degrees")

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Calculate the new bounding dimensions of the image
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust the rotation matrix to the center of rotation
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    # Perform the actual rotation and store the image
    rotated = cv2.warpAffine(image, M, (new_w, new_h))
    # output_path = f"output_{image_path.split('/')[-1]}"
    # cv2.imwrite(output_path, rotated)

    return rotated


In [7]:
def get_bound(image):
    res = reader.readtext(image)

    height,width=image.shape[:2]
    bound=int(width*0.75)
    cur=1
    texts=[]
    cropped_images = []
    for (bbox, text, prob) in res:
    # unpack the bounding box
        (tl, tr, br, bl) = bbox
        tl = (int(tl[0]), int(tl[1]))
        tr = (int(tr[0]), int(tr[1]))
        br = (int(br[0]), int(br[1]))
        bl = (int(bl[0]), int(bl[1]))

        if(cur==1 and len(text)>4 and ('/' in text) ):
            bound=tl[0]-50
            cur=0
            break
    return bound

def crop_image(image_path):

    image=correct_image_orientation(image_path)
    height,width=image.shape[:2]

    bound=get_bound(image[:height//15,:])
    cropped_image=image[:,bound:]
    return cropped_image

def blur_text_regions(image_path):
    # Read the image
    cropped=crop_image(image_path)
    image=cropped.copy()
    # Perform OCR to detect text regions
    results = reader.readtext(image)

    # Loop through the detected text regions
    for (bbox, text, prob) in results:
        # Extract the bounding box coordinates
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))

        # Extract the region of interest (ROI)
        roi = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        roi_height = bottom_right[1] - top_left[1]
        roi_width = bottom_right[0] - top_left[0]
        kernel_size = (max(1, roi_width // 3) | 1, max(1, roi_height // 3) | 1)
        # Apply Gaussian blur to the ROI
        blurred_roi = cv2.GaussianBlur(roi, kernel_size, 0)

        # Replace the original ROI with the blurred ROI
        image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = blurred_roi

    return image,cropped

In [8]:
def detect_lines(image_path):
    """
    Detect lines in an image using Hough Line Transform.

    Args:
    - image_path: Path to the input image.

    Returns:
    - image_with_lines: The image annotated with detected lines.
    - lines: The list of detected lines.
    """
    # # Load the image
    image,org_image = blur_text_regions(image_path);
    # Check if the image is loaded properly
    if image is None:
        raise ValueError(f"Image not found or could not be loaded from {image_path}")

    # Convert the image to grayscale if it's not already
    if len(image.shape) == 3:  # Check if the image has 3 channels (color image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image  # Image is already grayscale

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=90, minLineLength=90, maxLineGap=10)

    # Draw the lines on the image
    horizontal_lines = []
    if lines is not None:
        for line in lines:

            x1, y1, x2, y2 = line[0]
            if x1>image.shape[1]/3:
                    angle = np.arctan2(y2 - y1, x2 - x1) * 180.0 / np.pi
                    # Consider lines with an angle close to 0 or 180 degrees as horizontal
                    if abs(angle) < 60 or abs(angle - 180) < 60:


                            horizontal_lines.append(line)

    horizontal_lines.sort(key=lambda line: line[0][1])

    # Filter out lines that are too close to each other
    filtered_horizontal_lines = []
    for i in range(len(horizontal_lines)):
        if i == 0:
            filtered_horizontal_lines.append(horizontal_lines[i])
        else:
            _, y1_prev, _, y2_prev = horizontal_lines[i - 1][0]
            _, y1_curr, _, y2_curr = horizontal_lines[i][0]
            if abs(y1_curr - y1_prev) >=200 and abs(y2_curr - y2_prev) >= 200:
                filtered_horizontal_lines.append(horizontal_lines[i])
    filtered_horizontal_lines=[i for i in filtered_horizontal_lines if (i[0][1]>100 and i[0][1]<7000)]
    if(len(filtered_horizontal_lines)>0):
        temp=[]
        for i in range(0,len(filtered_horizontal_lines)-1):
            temp.append(filtered_horizontal_lines[i])
            if(filtered_horizontal_lines[i+1][0][1]-filtered_horizontal_lines[i][0][1]>1000):
                diff=filtered_horizontal_lines[i+1][0][1]-filtered_horizontal_lines[i][0][1]
                diff=diff//2
                temp.append([[filtered_horizontal_lines[i][0][0],filtered_horizontal_lines[i][0][1]+diff,filtered_horizontal_lines[i][0][2],filtered_horizontal_lines[i][0][3]+diff]])
        temp.append(filtered_horizontal_lines[-1])
    else:
        temp=filtered_horizontal_lines

    return org_image, temp
def divide_image_into_parts(image, lines):
    """
    Divide the image into 10 parts using the provided lines.

    Args:
    - image: The input image.
    - lines: The list of line coordinates.

    Returns:
    - divided_images: List of 10 image parts.
    """
    lines.sort(key=lambda line: line[0][1])  # Ensure lines are sorted by their y-coordinates

    divided_images = []
    height, width = image.shape[:2]

    # Add the top and bottom boundaries of the image to the lines
    all_y_coords =  [line[0][1] for line in lines]

    # Extract 10 parts of the image based on the y-coordinates
    for i in range(len(all_y_coords) - 1):
        y1 = all_y_coords[i]
        y2 = all_y_coords[i + 1]
        part = image[y1:y2, :]
        divided_images.append(part)

    return divided_images

In [21]:
def get_predicted_answers(image_path):
    org_image,lines=detect_lines(image_path)
    org_image=preprocess_image(org_image.copy())
    divided_images=divide_image_into_parts(org_image,lines)
    texts=[]
    for img in divided_images:
        results = reader.readtext(img)
        i=0;
        for bbox,text,score in results:
            if(len(text)==0):
              continue
            if(i!=0):
                break
            texts.append(txt(text))
            i+=1
        if(i==0):
            texts.append("nan")
    # texts=np.array(texts)
    return texts

In [22]:
def cal_marks(a1,a2):
    ans=0
    for i in range(len(a1)):
        if(a1[i].lower()==str(a2[i]).lower()):
            ans+=1
    return ans
def get_pred_marks(image_path,correct_answer):
    name=image_path.split("/")[-1]
    answer=get_predicted_answers(image_path)[:10]
    f=0
    if(len(answer)<10):
        f=1
        while len(answer)<10:
            answer.append("nan")
    pred=cal_marks(answer,correct_answer)
    if(f==1 and pred<2):
      return [name,5]
    else:
      return [name,pred]

In [9]:
import subprocess
zip_file="/content/gdrive/MyDrive/Sample_Data.zip"
passwd="akshat"
output_dir="/content/data1"
subprocess.run(["unzip", "-P", passwd, zip_file, "-d", output_dir])

CompletedProcess(args=['unzip', '-P', 'akshat', '/content/gdrive/MyDrive/Sample_Data.zip', '-d', '/content/data1'], returncode=0)

In [7]:
dir=output_dir+"/Sample_Data"
arr=[]
for root,dirs,files in os.walk(dir):
    for file in files:
        path=dir+"/"+file
        arr.append(path)
file_names=arr.copy()
correct_answer_df=pd.read_csv("/content/gdrive/MyDrive/auto-eval/data/ModelAnswer (1).csv")
correct_answer=correct_answer_df['Correct Answer']

In [16]:
get_pred_marks("/content/gdrive/MyDrive/auto-eval/data/Sample_Data/20240328_153807.jpg",correct_answer)

{'20240328_153807.jpg': 7}

In [24]:
predictions=[]
for path in tqdm(file_names):
    pred=get_pred_marks(path,correct_answer)
    predictions.append(pred)

100%|██████████| 48/48 [04:32<00:00,  5.69s/it]


,0,1
0,20240328_155745.jpg,6
1,20240328_153522.jpg,3
2,20240328_153456.jpg,6
3,20240328_151122.jpg,4
4,20240328_153252.jpg,5
5,20240328_150204.jpg,4
6,20240328_151046.jpg,4
7,20240328_153543.jpg,6
8,20240328_152349.jpg,4
9,20240328_145719.jpg,6


In [38]:
df=pd.DataFrame(predictions,columns=["Filename","Predicted Marks"]).set_index("Filename").sort_index() # predicted marks dataset
marks=pd.read_excel("/content/gdrive/MyDrive/auto-eval/data/Marks.xlsx").set_index("Filename").sort_index() #dataset for real marks for image
df.head()

,Predicted Marks
Filename,
20240328_145148.jpg,6
20240328_145219.jpg,3
20240328_145247.jpg,4
20240328_145458.jpg,8
20240328_145719.jpg,6


In [32]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(df,marks)
mae

0.8333333333333334